In [2]:
import numpy as np
import pandas as pd
import ast

In [64]:
movies_raw = pd.read_csv('dataset/movies.csv')
credits_raw = pd.read_csv('dataset/credits.csv')
ratings = pd.read_csv('dataset/ratings.csv')

In [65]:
movies = movies_raw.merge(credits_raw,on='title')

In [66]:
# movies.dropna()

In [67]:
# movies = pd.read_csv('movies_new.csv')

In [68]:
# movies.dropna()
# movies = movies.sort_values(by = 'vote_count',ascending=False)

In [69]:
# movies = movies.head(1500)

In [70]:
movies.head(20)

,movieId,tmdbId,budget,genres,homepage,id,keywords,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,1,97,160000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",http://inceptionmovie.warnerbros.com/,27205,"[{""id"": 1014, ""name"": ""loss of lover""}, {""id"":...",en,Inception,"Cobb, a skilled thief who commits corporate es...",...,148,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Your mind is the scene of the crime.,Inception,8.1,13752,27205,"[{""cast_id"": 1, ""character"": ""Dom Cobb"", ""cred...","[{""credit_id"": ""56e8462cc3a368408400354c"", ""de..."
1,2,66,185000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",http://thedarkknight.warnerbros.com/dvdsite/,155,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight,Batman raises the stakes in his war on crime. ...,...,152,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Why So Serious?,The Dark Knight,8.2,12002,155,"[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de..."
2,3,1,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
3,4,17,220000000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",http://marvel.com/avengers_movie/,24428,"[{""id"": 242, ""name"": ""new york""}, {""id"": 5539,...",en,The Avengers,When an unexpected enemy emerges and threatens...,...,143,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Some assembly required.,The Avengers,7.4,11776,24428,"[{""cast_id"": 46, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""52fe4495c3a368484e02b1cf"", ""de..."
4,5,789,58000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.foxmovies.com/movies/deadpool,293660,"[{""id"": 2095, ""name"": ""anti hero""}, {""id"": 307...",en,Deadpool,Deadpool tells the origin story of former Spec...,...,108,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Witness the beginning of a happy ending,Deadpool,7.4,10995,293660,"[{""cast_id"": 99, ""character"": ""Wade Wilson / D...","[{""credit_id"": ""56c986b2925141172f0068b6"", ""de..."
5,6,96,165000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",http://www.interstellarmovie.net/,157336,"[{""id"": 83, ""name"": ""saving the world""}, {""id""...",en,Interstellar,Interstellar chronicles the adventures of a gr...,...,169,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Mankind was born on Earth. It was never meant ...,Interstellar,8.1,10867,157336,"[{""cast_id"": 9, ""character"": ""Joseph Cooper"", ...","[{""credit_id"": ""52fe4bbf9251416c910e4801"", ""de..."
6,7,288,100000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 37, ""name...",http://unchainedmovie.com/,68718,"[{""id"": 801, ""name"": ""bounty hunter""}, {""id"": ...",en,Django Unchained,"With the help of a German bounty hunter, a fre...",...,165,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"Life, liberty and the pursuit of vengeance.",Django Unchained,7.8,10099,68718,"[{""cast_id"": 6, ""character"": ""Django"", ""credit...","[{""credit_id"": ""5703f6a2c3a36869850001d9"", ""de..."
7,8,95,170000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",http://marvel.com/guardians,118340,"[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...",en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",...,121,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,All heroes start 

In [71]:
# movies.to_csv('movies_new.csv', header=True, index=False, encoding='utf-8')

In [72]:
movies = movies[['movieId','genres','keywords','title','overview','cast','crew','tmdbId']]

In [73]:
movies.dropna(inplace=True)

In [74]:
movies.duplicated().sum()

0

In [75]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 53, "name": "Thriller"}, {"id": 878, "name": "Science Fiction"}, {"id": 9648, "name": "Mystery"}, {"id": 12, "name": "Adventure"}]'

In [76]:
'''
Content based generation
'''

'\nContent based generation\n'

In [77]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [78]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [79]:
def convertCast(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 3:
            L.append(i['name'])
            counter+=1
        else:
            break;
    return L

In [80]:
movies['cast'] = movies['cast'].apply(convertCast)

In [81]:
def fetchDirector(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break;
    return L

In [82]:
movies['crew'] = movies['crew'].apply(fetchDirector)

In [83]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [84]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [85]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [132]:
newDf = movies[['movieId','title','tags','tmdbId']]

In [133]:
newDf['tags'] = newDf['tags'].apply(lambda x: " ".join(x))

C:\Users\otbho\AppData\Local\Temp/ipykernel_14436/3762652423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(lambda x: " ".join(x))


In [134]:
newDf['tags'] = newDf['tags'].apply(lambda x: x.lower())

C:\Users\otbho\AppData\Local\Temp/ipykernel_14436/2614462281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(lambda x: x.lower())


In [135]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [136]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [137]:
newDf['tags'] = newDf['tags'].apply(stem)

C:\Users\otbho\AppData\Local\Temp/ipykernel_14436/877819958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDf['tags'] = newDf['tags'].apply(stem)


In [138]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [139]:
vectors = cv.fit_transform(newDf['tags']).toarray()

In [140]:
from sklearn.metrics.pairwise import cosine_similarity

In [141]:
similarity = cosine_similarity(vectors)

In [142]:
def recommend(movieId):
    idx = newDf[newDf['movieId'] == movieId].index[0]
    distances = similarity[idx]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    L = []
    for i in movies_list:
        L.append(newDf.iloc[i[0]].title)
    return L

In [146]:
recommend(21)

['The Lord of the Rings: The Two Towers',
 'The Hobbit: The Battle of the Five Armies',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Hobbit: An Unexpected Journey',
 'The Hobbit: The Desolation of Smaug',
 'Warcraft',
 'The Chronicles of Narnia: The Voyage of the Dawn Treader',
 'Brave',
 'Underworld: Rise of the Lycans',
 'The Scorpion King']

In [144]:
movies.head(10)

,movieId,genres,keywords,title,overview,cast,crew,tmdbId,tags
0,1,"[Action, Thriller, ScienceFiction, Mystery, Ad...","[lossoflover, dream, kidnapping, sleep, subcon...",Inception,"[Cobb,, a, skilled, thief, who, commits, corpo...","[LeonardoDiCaprio, JosephGordon-Levitt, EllenP...",[ChristopherNolan],97,"[Cobb,, a, skilled, thief, who, commits, corpo..."
1,2,"[Drama, Action, Crime, Thriller]","[dccomics, crimefighter, secretidentity, scare...",The Dark Knight,"[Batman, raises, the, stakes, in, his, war, on...","[ChristianBale, HeathLedger, AaronEckhart]",[ChristopherNolan],66,"[Batman, raises, the, stakes, in, his, war, on..."
2,3,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],1,"[In, the, 22nd, century,, a, paraplegic, Marin..."
3,4,"[ScienceFiction, Action, Adventure]","[newyork, shield, marvelcomic, superhero, base...",The Avengers,"[When, an, unexpected, enemy, emerges, and, th...","[RobertDowneyJr., ChrisEvans, MarkRuffalo]",[JossWhedon],17,"[When, an, unexpected, enemy, emerges, and, th..."
4,5,"[Action, Adventure, Comedy]","[antihero, mercenary, marvelcomic, superhero, ...",Deadpool,"[Deadpool, tells, the, origin, story, of, form...","[RyanReynolds, MorenaBaccarin, EdSkrein]",[TimMiller],789,"[Deadpool, tells, the, origin, story, of, form..."
5,6,"[Adventure, Drama, ScienceFiction]","[savingtheworld, artificialintelligence, fathe...",Interstellar,"[Interstellar, chronicles, the, adventures, of...","[MatthewMcConaughey, JessicaChastain, AnneHath...",[ChristopherNolan],96,"[Interstellar, chronicles, the, adventures, of..."
6,7,"[Drama, Western]","[bountyhunter, hero, plantation, society, frie...",Django Unchained,"[With, the, help, of, a, German, bounty, hunte...","[JamieFoxx, ChristophWaltz, LeonardoDiCaprio]",[QuentinTarantino],288,"[With, the, help, of, a, German, bounty, hunte..."
7,8,"[Action, ScienceFiction, Adventure]","[marvelcomic, spaceship, space, outerspace, or...",Guardians of the Galaxy,"[Light, years, from, Earth,, 26, years, after,...","[ChrisPratt, ZoeSaldana, DaveBautista]",[JamesGunn],95,"[Light, years, from, Earth,, 26, years, after,..."
8,9,"[ScienceFiction, Adventure, Fantasy]","[hallucination, dystopia, femaleprotagonist, b...",The Hunger Games,"[Every, year, in, the, ruins, of, what, was, o...","[JenniferLawrence, JoshHutcherson, LiamHemsworth]",[GaryRoss],427,"[Every, year, in, the, ruins, of, what, was, o..."
9,10,"[Action, Adventure, ScienceFiction, Thriller]","[future, chase, post-apocalyptic, dystopia, au...",Mad Max: Fury Road,"[An, apocalyptic, story, set, in, the, furthes...","[TomHardy, CharlizeTheron, NicholasHoult]",[GeorgeMiller],128,"[An, apocalyptic, story, set, in, the, furthes..."


In [145]:
from joblib import dump
dump(similarity, 'model/content_based.joblib')
dump(newDf, 'model/movie_df.joblib')

['model/movie_df.joblib']

In [100]:
# ratings.to_csv('ratings.csv')

In [101]:
'''
Top Recommendation
'''

'\nTop Recommendation\n'

In [102]:

def weighted_rating(v,m,R,C):
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [129]:
# pre processing
vote_count = (
    ratings
    .groupby('movieId',as_index=False)
    .agg( {'userId':'count', 'rating':'mean'} )
    )
vote_count.columns = ['movieId', 'vote_count', 'avg_rating']

# calcuate input parameters
C = np.mean(vote_count['avg_rating'])
m = np.percentile(vote_count['vote_count'], 70)
vote_count = vote_count[vote_count['vote_count'] >= m]
R = vote_count['avg_rating']
v = vote_count['vote_count']
vote_count['weighted_rating'] = weighted_rating(v,m,R,C)

# post processing
vote_count = vote_count.merge(movies_raw, on = ['movieId'], how = 'left')
popular_items = vote_count.loc[:,['movieId','title', 'overview','weighted_rating','tmdbId']]
popular_items = popular_items.sort_values('weighted_rating',ascending=False)
popular_items = popular_items.head(10)

In [130]:
from joblib import dump
dump(popular_items, 'model/top_rated.joblib')

['model/top_rated.joblib']

In [131]:
popular_items

,movieId,title,overview,weighted_rating,tmdbId
403,1422,The Texas Chain Saw Massacre,Five friends visiting their grandfather's hous...,3.499807,3160
317,1126,Closer,"A witty, romantic, and very dangerous love sto...",3.499807,1409
181,654,xXx,Xander Cage is your standard adrenaline junkie...,3.499807,569
151,547,Ace Ventura: Pet Detective,He's Ace Ventura: Pet Detective. Jim Carrey is...,3.499807,2817
106,402,John Carter,"John Carter is a war-weary, former military ca...",3.249710,5
105,401,Scott Pilgrim vs. the World,Scott Pilgrim is a film adaptation of the crit...,3.249710,412
376,1325,Wild Card,When a Las Vegas bodyguard with lethal skills ...,3.249710,1675
218,791,Trainwreck,Having thought that monogamy was never possibl...,3.249710,1372
110,419,Resident Evil,"When a virus leaks from a top-secret facility,...",3.249710,1396
94,356,Dark Shadows,Vampire Barnabas Collins is inadvertently free...,3.249710,134


In [106]:
'''
CF Recommendation
'''

'\nCF Recommendation\n'

In [107]:
genre = movies['genres']
# get all possible genre
all_genre = set()
for c in genre:
    for i in c:
        all_genre.update(c)
#     distinct_genre = genre[c].str.lower().str.strip().unique()
#     all_genre.update(distinct_genre)
# all_genre.remove(None)
all_genre = list(all_genre)

In [108]:
from joblib import dump
dump(all_genre, 'model/genres.joblib')

['model/genres.joblib']

In [119]:
# create item-genre matrix
item_genre_mat = movies[['movieId', 'genres']].copy()
# item_genre_mat['genres'] = item_genre_mat['genres'].str.lower().str.strip()

# OHE the genres column
# for genre in all_genre:
#     item_genre_mat[genre] = np.where(item_genre_mat['genres'].str.contains(genre), 1, 0)
# item_genre_mat = item_genre_mat.drop(['genres'], axis=1)
# item_genre_mat = item_genre_mat.set_index('movieId')

TypeError: 'list' object cannot be interpreted as an integer

In [109]:

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

NUM_USERS = 100
NUM_MOVIES = 1500


# preprocess data
row = ratings['userId']
col = ratings['movieId']
data = ratings['rating']

# init user-item matrix
mat = csr_matrix((data, (row, col)))
mat.eliminate_zeros()

# calculate sparsity
sparsity = float(len(mat.nonzero()[0]))
sparsity /= (mat.shape[0] * mat.shape[1])
sparsity *= 100
print(f'Sparsity: {sparsity:4.2f}%. This means that {sparsity:4.2f}% of the user-item ratings have a value.')

# compute similarity
item_corr_mat = cosine_similarity(mat.T)

Sparsity: 0.34%. This means that 0.34% of the user-item ratings have a value.


In [110]:
from joblib import dump
dump(item_corr_mat, 'model/cf_model.joblib')

['model/cf_model.joblib']

In [111]:
def top_k_items(item_id, top_k, corr_mat, map_name):
    # sort correlation value ascendingly and select top_k item_id
    top_items = corr_mat[item_id,:].argsort()[-top_k:][::-1] 
    top_items = [map_name[e] for e in top_items] 

    return top_items

In [112]:
# get top-k similar items
ind2name = {ind:name for ind,name in enumerate(item_genre_mat.index)}
name2ind = {v:k for k,v in ind2name.items()}

NameError: name 'item_genre_mat' is not defined

In [113]:
def getCfRecommendations(movieId):
#     if movie_cf_mat is None:      
#         r = open('model/cf_memory.pkl', 'rb')
#         movie_cf_mat = pickle.load(r)
#         print('loaded cf')
#         r.close()
    similar_items = top_k_items(name2ind[movieId],
                            top_k = 10,
                            corr_mat = item_corr_mat,
                            map_name = ind2name)
    if(movieId in similar_items):
        similar_items.remove(int(movieId))
    res = movies.loc[movies['movieId'].isin(similar_items)]
    return res

In [114]:
getCfRecommendations(1484)

NameError: name 'name2ind' is not defined

In [115]:
movies_raw

,movieId,tmdbId,budget,genres,homepage,id,keywords,original_language,original_title,overview,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,97,160000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",http://inceptionmovie.warnerbros.com/,27205,"[{""id"": 1014, ""name"": ""loss of lover""}, {""id"":...",en,Inception,"Cobb, a skilled thief who commits corporate es...",...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",14-07-2010,825532764,148,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Your mind is the scene of the crime.,Inception,8.1,13752
1,2,66,185000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",http://thedarkknight.warnerbros.com/dvdsite/,155,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight,Batman raises the stakes in his war on crime. ...,...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",16-07-2008,1004558444,152,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Why So Serious?,The Dark Knight,8.2,12002
2,3,1,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
3,4,17,220000000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",http://marvel.com/avengers_movie/,24428,"[{""id"": 242, ""name"": ""new york""}, {""id"": 5539,...",en,The Avengers,When an unexpected enemy emerges and threatens...,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",25-04-2012,1519557910,143,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Some assembly required.,The Avengers,7.4,11776
4,5,789,58000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.foxmovies.com/movies/deadpool,293660,"[{""id"": 2095, ""name"": ""anti hero""}, {""id"": 307...",en,Deadpool,Deadpool tells the origin story of former Spec...,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",09-02-2016,783112979,108,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Witness the beginning of a happy ending,Deadpool,7.4,10995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1496,768,60000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/xxxstateoft...,11679,"[{""id"": 521, ""name"": ""washington d.c.""}, {""id""...",en,xXx: State of the Union,"Ice Cube stars as Darius Stone, a thrill-seeki...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",27-04-2005,71073932,101,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Prepare for the next level,xXx: State of the Union,4.7,549
1496,1497,136,150000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 27, ""name...",http://www.thewolfmanmovie.com/,7978,"[{""id"": 494, ""name"": ""father son relationship""...",en,The Wolfman,"Lawrence Talbot, an American man on a visit to...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",11-02-2010,0,102,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,When the moon is full the legend comes to life,The Wolfman,5.5,549
1497,1498,920,90000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",NaN,2112,"[{""id"": 242, ""name"": ""new york""}, {""id"": 2671,...",en,Payback,"With friends like these, who needs enemies? Th...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",05-02-1999,161626121,100,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,No More Mr. Nice Guy.,Payback,6.7,548
1498,1499,1014,50000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 53, ""name...",NaN,181283,"[{""id"": 818

In [179]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import gc

# from collections import defaultdict

# # synthesize data
# NUM_USERS = 101
# NUM_ITEMS = 1501
# user_id = np.arange(start = 0, stop = NUM_USERS)
# item_id = np.arange(start = 0, stop = NUM_ITEMS)
# np.random.seed(42)

# user_item_dict = defaultdict(list)

# for id in user_id:
    
#     # random the number of item generation
#     # for each user, random 3 to 5 items to be rated.
#     num_rand_item = np.random.randint(low = 3, high = 8)

#     # random from the item_id
#     rand_items = np.random.choice(item_id, size = num_rand_item, replace = False)

#     # random rating for each itme_id
#     rand_rating = np.random.randint(low = 1, high = 5, size = num_rand_item)

#     # collect the user-item paris.
#     for uid, iid,rating in zip([id] * num_rand_item, rand_items, rand_rating):
#         user_item_dict['userid'].append(uid)
#         user_item_dict['movieId'].append(iid)
#         user_item_dict['rating'].append(rating)

# # prepare dataframe
# ratings = pd.DataFrame(user_item_dict)
# print("Rating Dataframe")
# ratings[['userid','movieId']] = ratings[['userid','movieId']].astype(str)
# display(ratings.head())

Rating Dataframe


,userid,movieId,rating
0,0,51,1
1,0,1465,4
2,0,184,3
3,0,1000,4
4,0,746,3


In [184]:
# ratings.to_csv('ratings.csv',index=False)